In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)
from utils.downloads import _prepare_cache

# Intersecção entre setores censitários (2022) e a área de 1km a partir de Parques Urbanos e Lineares (buff)

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

Eu interpretei o indicador como <=1km, não como <1km

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende dos arquivos resultantes dos notebooks '.buffed_parques_urb_line' e '../load_data/setores_censitario'

In [5]:
gdf_buffed_parq = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'area_1km_parques_urbanos_lineares.parquet'        
    )
)

In [6]:
gdf_setores22= gpd.read_parquet(
    os.path.join(
        assets_path,
        'setores_2022.parquet'
    )
)

In [7]:
gdf_setores22.shape

(27301, 27)

# Overlay setores por buff

Antes de interseccionar, precisaremos ter a informação da área da geometria original, para poder fazer o cálculo da porcentagem da área:

In [8]:
gdf_setores22['area_setor']=gdf_setores22['geometry'].area

Usando o método overlay, fazemos a intersecção, que exluirá todos os setores que não estão na área do buffer dos parques. Os que estiverem parte dentro e parte fora, serão cortados seguindo a geometria do buff.

In [9]:
overlay_buff_setores= gpd.overlay(
    gdf_setores22,
    gdf_buffed_parq,
    how='intersection',
    keep_geom_type=True    
)

In [10]:
overlay_buff_setores.shape

(17752, 28)

Para apagarmos as geometrias desprezíveis eu usarei o mesmo padrão que usei para quadras viárias e outras malhas das Áreas Verdes Públicas: desprezar as geometrias menores que 1% da área total do menor setor

In [11]:
overlay_buff_setores['area_recort']=overlay_buff_setores['geometry'].area

In [12]:
gdf_setores22.loc[gdf_setores22['area_setor']==gdf_setores22['area_setor'].min()]

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,geometry,area_setor
22732,355030878000084,Urbana,1,0,3550308,São Paulo,355030878,Sé,35503087800,None,...,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,283.0,130.0,"POLYGON ((333744.46 7394168.456, 333741.319 73...",246.391634


In [13]:
min_setor= float(gdf_setores22['area_setor'].min())
min_1pct = min_setor*0.01
irrelevants = overlay_buff_setores.loc[overlay_buff_setores['area_recort']<min_1pct]
irrelevants.shape

(16, 29)

In [14]:
overlay_buff_setores=overlay_buff_setores.loc[overlay_buff_setores['area_recort']>=min_1pct]
overlay_buff_setores.shape

(17736, 29)

In [15]:
setores_omitidos = gdf_setores22.loc[~gdf_setores22['CD_SETOR'].isin(overlay_buff_setores['CD_SETOR'])]
setores_omitidos
print(f'{len(setores_omitidos)} setores foram omitidos após a intersecção')
print(f'{len(gdf_setores22)-len(setores_omitidos)} setores interseccionam com o buff dos parques')

9590 setores foram omitidos após a intersecção
17711 setores interseccionam com o buff dos parques


# Calcular a população

Como não temos como saber precisamente onde as pessoas se encontram dentro do setore censitário, estabelecemos por padrão que faremos o cálculo como se elas estivessem distribuídas de forma homogênea no espaço. A partir disso, podemos assumir que a maior % de área do setor, também equivalerá a um recorte com a maior quantidade de pessoas do setor, apesar de isso não necessariamente acontecer na realidade.

Para calcular a porcentagem da área do recorte, vamos calcular a área do recorte a partir da geometria atual, e dividí-la pela área total do setor, anteriormente calculada:
$$ (\text{Porcentagem da área do setor})=\frac{(\text{área do recorte do setor censitário})}{(\text{área total do setor censitário})}$$ 

Agora que sabemos a porcentagem da área de cada recorte, podemos multiplicá-la (já que está em decimal) pelo total da população no setor e teremos a quantidade de pessoas nesta "fatia". Repare que, por estarmos calculando o número de pessoas, é necessário que seja um número inteiro (ou um número arredondado a 0 casa decimais):
$$ \text{Quantidade de pessoas no recorte do setor} = \text{quantidade total de pessoas em todo o setor}*\text{porcentagem da área do setor}$$

In [16]:
overlay_buff_setores['percent_area'] = overlay_buff_setores['area_recort']/overlay_buff_setores['area_setor']

In [17]:
overlay_buff_setores['pop_recort']=round(overlay_buff_setores['total_pop']*overlay_buff_setores['percent_area'], 0)

Como esse método de arredondamento não leva em conta os outros recortes para arredondar, é possível que o número de pessoas varie.

In [18]:
overlay_buff_setores.sample(2)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,geometry,area_recort,percent_area,pop_recort
4390,355030828000175,Urbana,1,1,3550308,São Paulo,355030828,Ermelino Matarazzo,35503082800,None,...,São Paulo,3550308,São Paulo/SP,191.0,87.0,4980.570719,"POLYGON ((347876.394 7401071.328, 347873.532 7...",116.392097,0.023369,4.0
10624,355030860000299,Urbana,1,0,3550308,São Paulo,355030860,Perdizes,35503086000,None,...,São Paulo,3550308,São Paulo/SP,303.0,128.0,93676.048731,"POLYGON ((329108.162 7394773.462, 329096.856 7...",93676.048731,1.000000,303.0


Para visualizar melhor, vamos procurar só os casos que o setor não está completo:

In [19]:
overlay_buff_setores.loc[overlay_buff_setores['percent_area']<=0.99].sample(4)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,geometry,area_recort,percent_area,pop_recort
2108,355030817000387,Urbana,1,0,3550308,São Paulo,355030817,Campo Limpo,35503081700,None,...,São Paulo,3550308,São Paulo/SP,337.0,147.0,24616.716108,"POLYGON ((318341.639 7383562.101, 318329.851 7...",279.976184,0.011373,4.0
2906,355030820000037,Urbana,1,0,3550308,São Paulo,355030820,Carrão,35503082000,None,...,São Paulo,3550308,São Paulo/SP,619.0,272.0,53191.468733,"POLYGON ((343126.744 7394050.117, 343130.168 7...",23.778934,0.000447,0.0
6185,355030835000110,Urbana,1,0,3550308,São Paulo,355030835,Itaim Bibi,35503083500,None,...,São Paulo,3550308,São Paulo/SP,371.0,224.0,16377.669726,"POLYGON ((329357.244 7390929.291, 329269.791 7...",8021.777967,0.489800,182.0
3063,355030821000074,Urbana,1,0,3550308,São Paulo,355030821,Casa Verde,35503082100,None,...,São Paulo,3550308,São Paulo/SP,464.0,209.0,29460.756173,"POLYGON ((330125.898 7401316.028, 330134.164 7...",812.375366,0.027575,13.0


# Salvar gdf

In [20]:
save_parquet_excel(
    overlay_buff_setores,
    fname='overlay_setores22_buff_parques_urb_line',
    data_path = assets_path,
    data_subpath = 'parques_lineares_e_urbanos'
)